In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save
from transformations import hitnet_trafo

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols','/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [4]:
hits_input = tf.keras.Input(shape=(data.single_hits.shape[1],))
params_input = tf.keras.Input(shape=(data.repeated_params.shape[1],))
t = hitnet_trafo(labels=data.labels)

h = t(hits_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(1024, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

hit_net = tf.keras.Model(inputs=[hits_input, params_input], outputs=outputs)
hit_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo (hitnet_trafo)     (None, 17)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           576         hitnet_trafo[0][0]               
______________________________________________________________________________________________

In [5]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hit_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [6]:
train_id = 'HitNet_logE' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [7]:
callbacks = []
callbacks.append(Save(save_every=2, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [8]:
train_data, test_data = data.get_hitnet_data(train_batch_size=4096,
                                             test_batch_size=4096,
                                             )

In [ ]:
hist = hit_net.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                  initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
33572/33572 - 1267s - loss: 0.3367 - val_loss: 0.3163
Epoch 2/1000
33572/33572 - 1361s - loss: 0.3119 - val_loss: 0.3027
Epoch 3/1000
33572/33572 - 1171s - loss: 0.2993 - val_loss: 0.2899
Epoch 4/1000
33572/33572 - 1334s - loss: 0.2871 - val_loss: 0.2753
Epoch 5/1000


In [ ]:
# L7 0.28
# L3 0.239
# L3 with logE, angle correction, and pulse flags 0.205